In [5]:
import os
import pandas
import spacy
import torch
from  torch.nn.utils.rnn import pack_sequence
from torch.utils.data import DataLoader,Dataset
from PIL import Image



In [ ]:
class FlickrDataset(Dataset):
    def __init__(self,root_dir, caption_file,transform=None,freq_threshold=5) :
        self.root_dir =root_dir
        self.caption_dataFrame = caption_file
        self.transfroms= transform
        #getting images and their captions
        self.imgs = self.caption_dataFrame["image"]
        self.captions = self.caption_dataFrame["captions"]

        #initialize vocalary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.biuld_vocabulary(self.captions.tolist())
    
    def __len__(self):
        return len(self.caption_dataFrame)
    
    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir,img_id)).convert("RGB")
        
        if self.transfroms is not None:
            img = self.transfroms(img)
        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalized_caption(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)

        